In [15]:
import urllib
import os
import requests
import json

from collections import defaultdict, namedtuple
import urllib.request

import numpy as np
import pandas as pd

import jsonlines
import json
import time
from tqdm import tqdm
from numpy import dot
from numpy.linalg import norm
import statistics

In [16]:
def get_paper_by_ids(paperIds):
    r = requests.post(
        'https://api.semanticscholar.org/graph/v1/paper/batch',
        params={'fields': 'citationCount,title,corpusId'},
        json={"ids": paperIds}
    )
    return(r.json())

In [17]:
# Helpers from https://github.com/kwchurch/JSALT_Better_Together/blob/main/src/create_rotation_matrix.py

def record_size_from_dir(dir):
    with open(dir + '/record_size', 'r') as fd:
        return int(fd.read().split('\t')[0])

def map_from_dir(dir):
    fn = dir + '/map.old_to_new.i'
    fn_len = os.path.getsize(fn)
    return np.memmap(fn, dtype=np.int32, shape=(int(fn_len/4)), mode='r')

def embedding_from_dir(dir, K):
    fn = dir + '/embedding.f'
    fn_len = os.path.getsize(fn)
    return np.memmap(fn, dtype=np.float32, shape=(int(fn_len/(4*K)), K), mode='r')

def directory_to_config(dir):
    K = record_size_from_dir(dir)
    return { 'record_size' : K,
             'dir' : dir,
             'map' : map_from_dir(dir),
             'embedding' : embedding_from_dir(dir, K)}

In [18]:
specter = '/Volumes/kwc_JSALT/JSALTdir/semantic_scholar/embeddings/specter'
proposed = '/Volumes/kwc_JSALT/JSALTdir/semantic_scholar/embeddings/proposed'

In [19]:
inputs = specter, proposed

configs = [directory_to_config(d) for d in inputs]

map0 = configs[0]['map']
emb0 = configs[0]['embedding']

map1 = configs[1]['map']
emb1 = configs[1]['embedding']

In [20]:
print(map1[8])

5


In [21]:
t0 = time.time() # measure wall time
print(len(emb1[map1[278919]]))
print(len(emb0[map0[278919]]))
print(format(time.time() - t0, ".3f"), " seconds wall time to fetch specter & proposed.")

280
768
0.000  seconds wall time to fetch specter & proposed.


In [22]:
PATH = '/Users/shabnamtafreshi/Desktop/JSULT/paper_reviewer_matching/'
softFile = 'test_soft_qrel'
hardFile = 'test_hard_qrel'
reviewer_metafile = 'reviewer_metadata'

In [ ]:
def gen_corpusId_potential_reviewrs():
    t0 = time.time() # measure wall time
    specter_dataset: dict
    proposed_dataset = {}
    corpusId_potential_reviewrs = {}
    files = [softFile, hardFile]
    for file in files:
        with jsonlines.open(PATH + file + '.jsonl') as reader:
            for obj in reader:
                if 'icip' not in str(obj["query_id"]) and int(obj["query_id"]) not in specter_dataset.keys():
                    specter_dataset[obj["query_id"]] = emb0[map0[int(obj["query_id"])].tolist()]
                    proposed_dataset[obj["query_id"]] = emb1[map1[int(obj["query_id"])].tolist()]
                if 'icip' not in str(obj["query_id"]):
                    if str(obj["query_id"]) not in corpusId_potential_reviewrs.keys():
                        corpusId_potential_reviewrs[obj["query_id"]] = obj['cand_id']
                    else:
                        corpusId_potential_reviewrs[obj["query_id"]] += ',' + obj['cand_id']
    print(format(time.time() - t0, ".3f"), " seconds wall time to fetch specter & proposed.")
    return specter_dataset, proposed_dataset, corpusId_potential_reviewrs

In [ ]:
specter_dataset, proposed_dataset, corpusId_potential_reviewrs = gen_corpusId_potential_reviewrs(softFile, hardFile)

In [23]:
t0 = time.time() # measure wall time
specter_dataset = {}
proposed_dataset = {}
corpusId_potential_reviewrs = {}
files = [softFile, hardFile]
for file in files:
    with jsonlines.open(PATH + file + '.jsonl') as reader:
        for obj in reader:
            if 'icip' not in str(obj["query_id"]) and int(obj["query_id"]) not in specter_dataset.keys():
                specter_dataset[obj["query_id"]] = emb0[map0[int(obj["query_id"])].tolist()]
                proposed_dataset[obj["query_id"]] = emb1[map1[int(obj["query_id"])].tolist()]
            if 'icip' not in str(obj["query_id"]):
                if str(obj["query_id"]) not in corpusId_potential_reviewrs.keys():
                    corpusId_potential_reviewrs[obj["query_id"]] = obj['cand_id']
                else:
                    corpusId_potential_reviewrs[obj["query_id"]] += ',' + obj['cand_id']
print(format(time.time() - t0, ".3f"), " seconds wall time to fetch specter & proposed.")

0.024  seconds wall time to fetch specter & proposed.


In [24]:
print(len(specter_dataset))
print(len(proposed_dataset))

34
34


In [29]:
r_meta_data = []
reviewer_readfile = PATH + reviewer_metafile + '.jsonl'
with jsonlines.open(reviewer_readfile) as readR:
    for obj in readR:
        items = get_paper_by_ids(obj['papers'])
        temp_data = {"r_id": obj["r_id"], 'papers': items}
        r_meta_data.append(temp_data)

In [15]:
with open(PATH + reviewer_metafile + '_papers_meta.json', 'w', encoding='utf-8') as writefile:
    json.dump(r_meta_data, writefile, ensure_ascii=False, indent=4)

In [30]:
reviewers_hash = {}
for r_meta in r_meta_data:
    reviewers_hash[r_meta['r_id']] = r_meta['papers']

In [31]:
print(specter_dataset['1446998'])
#print(potential_reviewrs['777798'])

[-4.76485777e+00 -5.82242393e+00 -1.37443289e-01  9.44825172e-01
 -2.12372780e+00 -1.63317740e+00  4.33732605e+00 -2.78514171e+00
 -1.05489695e+00  6.34684384e-01 -1.70956481e+00  6.30087280e+00
 -2.41054630e+00  7.20137358e-01 -4.11513758e+00  1.43669200e+00
  7.45488882e-01  2.42300987e+00  1.41638947e+00  2.66247034e+00
 -1.24050117e+00  3.50233293e+00 -3.86009359e+00 -1.53281081e+00
 -2.41259003e+00 -6.54624879e-01  2.07140541e+00  3.09908295e+00
 -3.85731697e-01 -2.12392378e+00  2.00849581e+00 -3.15520048e+00
  5.53929901e+00 -5.59345007e-01  1.47879338e+00 -8.52572024e-01
  9.57863092e-01  7.26139927e+00 -3.88091779e+00 -1.37052333e+00
 -2.47357321e+00  3.45721245e+00  4.62212533e-01 -4.96912122e-01
  1.58963621e+00 -1.08264661e+00 -1.77406991e+00  5.39235020e+00
 -4.99304891e-01 -9.99966204e-01  2.22011948e+00  1.48010421e+00
  6.24849081e-01 -8.43723893e-01  2.98292017e+00 -3.66626120e+00
  6.69024348e-01  2.20757246e+00  3.18392181e+00 -2.78258872e+00
  2.49615145e+00  4.41709

In [35]:
final_results = []
foo = 0
bar = 0
for corpusId, potential_reviewrs in corpusId_potential_reviewrs.items():
    parts = potential_reviewrs.split(',')
    for reviewerID in parts:
        #print(reviewerID)
        specter_cosines = []
        proposed_cosines = []
        counter = 0
        if reviewerID in reviewers_hash.keys():
            for k in range(len(reviewers_hash[reviewerID])):
                #print(reviewers_hash[reviewerID][k]['corpusId'])
                if reviewers_hash[reviewerID][k] is not None and int(reviewers_hash[reviewerID][k]['corpusId']) in map0:
                    #print(reviewers_hash[reviewerID][k]['corpusId'])
                    specter_dataset[reviewers_hash[reviewerID][k]['corpusId']] = emb0[map0[int(reviewers_hash[reviewerID][k]['corpusId'])].tolist()]
                    proposed_dataset[reviewers_hash[reviewerID][k]['corpusId']] = emb1[map1[int(reviewers_hash[reviewerID][k]['corpusId'])].tolist()]
                    # Calculating cosign similarity
                    specter_cosines.append(dot(specter_dataset[corpusId], specter_dataset[reviewers_hash[reviewerID][k]['corpusId']])/(norm(specter_dataset[corpusId])*norm(specter_dataset[reviewers_hash[reviewerID][k]['corpusId']])))
                    proposed_cosines.append(dot(proposed_dataset[corpusId], proposed_dataset[reviewers_hash[reviewerID][k]['corpusId']])/(norm(proposed_dataset[corpusId])*norm(proposed_dataset[reviewers_hash[reviewerID][k]['corpusId']])))
                    counter += 1

            if len(specter_cosines) < 3:
                num_of_papers = len(specter_cosines)
                foo += 1
            else:
                num_of_papers = 3
                bar += 1

            if len(specter_cosines) == 0:
                mean_specter_cosines = 0
                mean_proposed_cosines = 0

            else:
                mean_specter_cosines = statistics.mean(sorted(specter_cosines[:num_of_papers]))
                mean_proposed_cosines = statistics.mean(sorted(proposed_cosines[:num_of_papers]))
            final_results.append({'corpusId': corpusId, 'r_id': reviewerID,
                                  'specter_score': str(mean_specter_cosines), 'proposed_score': str(mean_proposed_cosines),
                                  'numOfPapers': counter, 'numOfPapersWEmbed': num_of_papers})

In [36]:
print(len(final_results))
print(foo)
print(bar)

2092
28
2064


In [37]:
with open(PATH + reviewer_metafile + '_papers_meta_scores.json', 'w', encoding='utf-8') as writefile:
    json.dump(final_results, writefile, ensure_ascii=False, indent=4)